In [1]:
import pandas as pd
import numpy as np
from scipy import sparse

from copy import deepcopy


/home/ag16115/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


# finished functions 

In [2]:
def graph_from_dataframe(data, partition_pairs, join_token='::'):
    """
    Create a DMP graph from a dataframe.

    Parameters
    ----------
    data : pandas.DataFrame or list of pandas.DataFrame
        The data to be used to create the graph. If a list of dataframes is
        passed, each dataframe is used to create a separate graph.
    partition_pairs : list of lists
        The partition pairs to be used to create the graph. Each element of
        the list is a list of two elements, which are the names of the
        partitions to be joined.
    join_token : str
        The token used to join the partition name and the node name.

    Returns
    -------
    A : scipy.sparse.csr_matrix
        The adjacency matrix of the graph.
    attributes : list of lists
        The attributes of the nodes. The first list contains the attributes
        of the nodes that do not change over time. The second list contains
        the attributes of the nodes that change over time.
    """
    if not isinstance(data, list):
        data = [data]
    if not isinstance(partition_pairs[0][0], list):
        partition_pairs = [partition_pairs]
        
    edge_list = []
    for data0, partition_pairs0 in zip(data, partition_pairs):
        for partition_pair in partition_pairs0:
            pair_data = data0[partition_pair].drop_duplicates()
            pair_data.columns = ['V1', 'V2']
            pair_data['V1'] = [partition_pair[0] + join_token + str(x) for x in pair_data['V1']]
            pair_data['V2'] = [partition_pair[1] + join_token + str(x) for x in pair_data['V2']]
            pair_data['P1'] = partition_pair[0]
            pair_data['P2'] = partition_pair[1]
            edge_list.append(pair_data)
            
    edge_list = pd.concat(edge_list)
    nodes = sorted(list(set(list(edge_list['V1']) + list(edge_list['V2']))))
    node_ids = {x: i for i, x in enumerate(nodes)}
    partitions = [str.split(x, join_token)[0] for x in nodes]
    n_nodes = len(nodes)
    
    edge_list['V_ID1'] = [node_ids[x] for x in edge_list['V1']]
    edge_list['V_ID2'] = [node_ids[x] for x in edge_list['V2']]
    
    A = sparse.coo_matrix((np.ones(2*len(edge_list)),
                    (pd.concat([edge_list['V_ID1'], edge_list['V_ID2']]),
                     pd.concat([edge_list['V_ID2'], edge_list['V_ID1']]))),
                    shape=[n_nodes, n_nodes])
    
    attributes = [[{'name': name, 'partition': partition} for name, partition in zip(nodes, partitions)]] * 2

    A = A.tocsr()
    return A, attributes

In [3]:
# realise this may seem like a convoluted way to do this but it'll make it easier to add time element later
def find_subgraph(A, attributes, subgraph_attributes):
    """
    Find a subgraph of a multipartite graph.

    Parameters
    ----------
    A : scipy.sparse.csr_matrix
        The adjacency matrix of the multipartite graph.
    attributes : list of lists
        The attributes of the nodes. The first list contains the attributes
        of the nodes in rows. The second list contains
        the attributes of the nodes in the columns.
    subgraph_attributes : list of lists
        The attributes of the nodes of the wanted in the subgraph. The first list contains
        the attributes of the nodes wanted in the rows. The second
        list contains the attributes of the nodes wanted in the column.

    Returns
    -------
    subgraph_A : scipy.sparse.csr_matrix
        The adjacency matrix of the subgraph.
    subgraph_attributes : list of lists
        The attributes of the nodes of the subgraph. The first list contains
        the attributes of the nodes in the rows. The second
        list contains the attributes of the nodes in the columns.
    """

    if not isinstance(subgraph_attributes[0], list):
        subgraph_attributes[0] = [subgraph_attributes[0]]

    if not isinstance(subgraph_attributes[1], list):
        subgraph_attributes[1] = [subgraph_attributes[1]]

    # find the indices of the rows with required attributes
    subgraph_node_indices_row = []
    for node_idx, node_attributes in enumerate(attributes[0]):
        for each_subgraph_attributes in subgraph_attributes[0]:
            matched = True
            for key, value in each_subgraph_attributes.items():
                if key not in node_attributes or node_attributes[key] != value:
                    matched = False
                    break
            if matched:
                subgraph_node_indices_row.append(node_idx)

    # find the indices of the columns with required attributes
    subgraph_node_indices_col = []
    for node_idx, node_attributes in enumerate(attributes[1]):
        for each_subgraph_attributes in subgraph_attributes[1]:
            matched = True
            for key, value in each_subgraph_attributes.items():
                if key not in node_attributes or node_attributes[key] != value:
                    matched = False
                    break
            if matched:
                subgraph_node_indices_col.append(node_idx)

    # create subgraph and subgraph attributes
    subgraph_A = A[np.ix_(subgraph_node_indices_row,
                          subgraph_node_indices_col)]
    subgraph_attributes = [[attributes[0][i] for i in subgraph_node_indices_row], [
        attributes[1][i] for i in subgraph_node_indices_col]]

    return subgraph_A, subgraph_attributes

In [58]:
def zero_matrix(m, n = None):
    """
    Create a zero matrix.
    """
    if n == None:
        n = m
    M = sparse.coo_matrix(([],([],[])),shape = (m,n))
    return M

def symmetric_dilation(M):
    """
    Dilate a matrix to a symmetric matrix.
    """
    m, n = M.shape
    D = sparse.vstack([sparse.hstack([zero_matrix(m), M]), sparse.hstack([M.T, zero_matrix(n)])])
    return D

def find_connected_components(A, attributes, n_components = 1):
    """
    Find connected components of a multipartite graph.

    Parameters
    ----------
    A : scipy.sparse.csr_matrix
        The adjacency matrix of the graph.
    attributes : list of lists
        The attributes of the nodes. The first list contains the attributes
        of the nodes in rows. The second list contains
        the attributes of the nodes in the columns.
    n_components : int
        The number of components to be found.

    Returns
    -------
    cc_As : list of scipy.sparse.csr_matrix
        The adjacency matrices of the connected components.
    cc_attributes : list of lists
        The attributes of the nodes of the connected components. The first list contains
        the attributes of the nodes in the rows. The second
        list contains the attributes of the nodes in the columns.
    """
    
    A_dilation = symmetric_dilation(A)
    _, cc = connected_components(A_dilation)
    cc = [cc[:A.shape[0]], cc[A.shape[0]:]]
    if n_components == None:
        n_components = np.max(cc) + 1
    if _ == 1:
        return A, attributes
    else:
        cc_As = []
        cc_attributes = []
        for i in range(n_components):
            idx0 = np.where(cc[0] == i)[0]
            idx1 = np.where(cc[1] == i)[0]
            store_cc_A, store_cc_attributes = subgraph_idx(A,attributes, idx0, idx1)
            cc_As.append(store_cc_A)
            cc_attributes.append(store_cc_attributes)
        return cc_As, cc_attributes


# need to sort out

In [4]:
def safe_inv_sqrt(a, tol=1e-12):
    """
    Compute the inverse square root of an array, ignoring division by zero.
    """
    with np.errstate(divide="ignore"):
        b = 1 / np.sqrt(a)
    b[np.isinf(b)] = 0
    b[a < tol] = 0
    return b

def to_laplacian(A, regulariser=0):
    """
    Convert an adjacency matrix to a Laplacian matrix.

    Parameters
    ----------
    A : scipy.sparse.csr_matrix
        The adjacency matrix.
    regulariser : float
        The regulariser to be added to the degrees of the nodes.

    Returns
    -------
    L : scipy.sparse.csr_matrix
        The Laplacian matrix.
    """

    left_degrees = np.reshape(np.asarray(A.sum(axis=1)), (-1,))
    right_degrees = np.reshape(np.asarray(A.sum(axis=0)), (-1,))
    if regulariser == 'auto':
        regulariser = np.mean(np.concatenate((left_degrees, right_degrees)))
    left_degrees_inv_sqrt = safe_inv_sqrt(left_degrees + regulariser)
    right_degrees_inv_sqrt = safe_inv_sqrt(right_degrees + regulariser)
    L = sparse.diags(left_degrees_inv_sqrt) @ A @ sparse.diags(right_degrees_inv_sqrt)
    return L

In [5]:
def embed(G, d = 10, matrix = 'adjacency', regulariser = 0):
    if matrix == 'laplacian':
        L = to_laplacian(G.A, regulariser)
        u, s, vT = svds(L, d)
    else:
        u, s, vT = svds(G.A, d)
    o = np.argsort(s[::-1])
    X = u[:,o] @ np.diag(np.sqrt(s[o]))
    Y = vT.T[:,o] @ np.diag(np.sqrt(s[o]))
    left_embedding = Embedding(X,
                               attributes = deepcopy(G.attributes[0]),
                               metadata = {'singular values': s[o]})
    right_embedding = Embedding(Y,
                                attributes = deepcopy(G.attributes[1]),
                                metadata = {'singular values': s[o]})                                  
    return left_embedding, right_embedding

In [6]:
def truncate(X, d, inplace=False):
    if inplace:
        Y = X
    else:
        Y = deepcopy(X)
    Y.X = Y.X[:, :d]
    return Y

def select(embedding, attributes, inplace=False):
    if not isinstance(attributes, list):
        attributes = [attributes]
    which_nodes = list()
    for attributes_dict in attributes:
        for a, v in attributes_dict.items():
            if not isinstance(v, list):
                v = [v]
        which_nodes_by_attribute = [[i for i, y in enumerate(embedding.attributes) if y[a] in v] for a, v in attributes_dict.items()]
        which_nodes.append(list(set.intersection(*map(set, which_nodes_by_attribute))))
    which_nodes = list(set().union(*which_nodes))
    if inplace:
        embedding.X = embedding.X[which_nodes, :]
        embedding.attributes = [embedding.attributes[i] for i in which_nodes]
        return None
    else:
        selected_X = embedding.X[which_nodes, :]
        selected_attributes = [embedding.attributes[i] for i in which_nodes]
        selected = Embedding(deepcopy(selected_X), deepcopy(selected_attributes), deepcopy(embedding.metadata))
        return selected

def recover_subspaces(embedding):    
    partitions = list(set([x['partition'] for x in embedding.attributes]))
    partition_embeddings = {}
    for p in partitions:
        p_embedding = select(embedding, {'partition': p})
        Y = p_embedding.X
        u, s, vT = linalg.svd(Y, full_matrices=False)
        o = np.argsort(s[::-1])
        Y = Y @ vT.T[:, o]
        p_embedding.X = Y
        p_embedding.metadata['partition singular values'] = s[o]
        partition_embeddings[p] = p_embedding
    return partition_embeddings

def degree_correction(X, inplace=False):
    tol = 1e-12
    if inplace:
        Y = X
    else:
        Y = deepcopy(X)
    norms = np.linalg.norm(Y.X, axis=1)
    idx = np.where(norms > tol)
    Y.X[idx] = Y.X[idx] / (norms[idx, None])
    return Y

class Embedding:
    def __init__(self, X, attributes={}, metadata={}):
        self.X = X
        self.attributes = attributes
        self.metadata = metadata

In [24]:
import networkx as nx
from scipy.sparse import coo_matrix
from scipy.sparse.csgraph import connected_components
from copy import deepcopy

def subgraph_idx(A, attributes, idx0, idx1):
    """
    Find a subgraph of a multipartite graph by indices.
    """  
    subgraph_A = A[np.ix_(idx0, idx1)]
    subgraph_attributes = [
        [attributes[0][i] for i in idx0],
        [attributes[1][i] for i in idx1]
    ]
    return subgraph_A, subgraph_attributes

def symmetric_dilation(A):
    """
    Dilate a graph to a symmetric graph.

    Parameters
    ----------
    A : scipy.sparse.csr_matrix
        The adjacency matrix of the graph.
        
    Returns
    -------
    A_dilation : scipy.sparse.csr_matrix
        The adjacency matrix of the symmetric graph.
    """

    return A + A.T

# def connected_components(A, n_components = 1):
#     A_dilation = symmetric_dilation(A)
#     _, cc = connected_components(A_dilation)
#     cc = [cc[:A.shape[0]], cc[A.shape[0]:]]
#     if n_components == None:
#         n_components = np.max(cc) + 1
#     Gcc = []
#     for i in range(n_components):
#         idx0 = np.where(cc[0] == i)[0]
#         idx1 = np.where(cc[1] == i)[0]
#         Gcc.append(subgraph_idx(A, idx0, idx1))
#     return Gcc


def to_networkx(A, attributes, symmetric=None):
    if symmetric == None:
        if is_symmetric(A):
            symmetric = True
        else:
            symmetric = False
    if symmetric:
        G_nx = nx.Graph(A)
        nx.set_node_attributes(G_nx, {i: a for i, a in enumerate(attributes[0])})
        return G_nx
    else:
        n0 = len(attributes[0])
        n1 = len(attributes[1])
        G_nx = nx.Graph(symmetric_dilation(A))
        nx.set_node_attributes(G_nx, {i: a for i, a in enumerate(attributes[0])})
        nx.set_node_attributes(G_nx, {i + n0: a for i, a in enumerate(attributes[1])})
        nx.set_node_attributes(G_nx, {i: {'bipartite': 0} for i in range(n0)})
        nx.set_node_attributes(G_nx, {i + n0: {'bipartite': 1} for i in range(n1)})
        return G_nx

# data

In [8]:
data = pd.read_csv('/home/ag16115/Documents/phd/codebase_data/activity_data.csv', sep = '\t', on_bad_lines='skip')

/tmp/ipykernel_24716/4058704425.py:1: DtypeWarning: Columns (13,16) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/home/ag16115/Documents/phd/codebase_data/activity_data.csv', sep = '\t', on_bad_lines='skip')


In [9]:
A, attributes = graph_from_dataframe(data, [['Company', 'Tender'],['Company', 'Buyer'],['Company', 'Item']], join_token='::')

In [10]:
# subgraph_attributes = [
#     [{'partition': 'Company'},{'partition': 'Buyer'}],
#     {'partition': 'Buyer'}
# ]

subgraph_attributes = [
    {'partition': 'Company'},
    {'partition': 'Buyer'}
]
subgraph_A, subgraph_attributes  = find_subgraph(A, attributes,subgraph_attributes)

In [15]:
A_dilation = symmetric_dilation(subgraph_A)

In [57]:
cc_A, cc_attributes = find_connected_components(subgraph_A, subgraph_attributes)